<a href="https://colab.research.google.com/github/gulabpatel/PySpark/blob/main/DataFrame_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark DataFrame Basics

Spark DataFrames are the workhouse and main way of working with Spark and Python post Spark 2.0. DataFrames act as powerful versions of tables, with rows and columns, easily handling large datasets. The shift to DataFrames provides many advantages:
* A much simpler syntax
* Ability to use SQL directly in the dataframe
* Operations are automatically distributed across RDDs
    
If you've used R or even the pandas library with Python you are probably already familiar with the concept of DataFrames. Spark DataFrame expand on a lot of these concepts, allowing you to transfer that knowledge easily by understanding the simple syntax of Spark DataFrames. Remember that the main advantage to using Spark DataFrames vs those other programs is that Spark can handle data across many RDDs, huge data sets that would never fit on a single computer. That comes at a slight cost of some "peculiar" syntax choices, but after this course you will feel very comfortable with all those topics!

Let's get started!

## Creating a DataFrame

First we need to start a SparkSession:

In [3]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 61kB/s 
     |████████████████████████████████| 204kB 39.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=c35f66940cd44d01b9eb1932c6a05a4bbbfe9f6602dca62b30e5e8bc90c9ac2d
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

Then start the SparkSession

In [6]:
# May take a little while on a local computer
spark = SparkSession.builder.appName("Basics").getOrCreate()

In [7]:
spark

In [8]:
# We'll discuss how to read other options later.
# This dataset is from Spark's examples

# Might be a little slow locally
df = spark.read.json('/content/people.json')

#### Showing the data

In [9]:
# Note how data is missing!
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [10]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [11]:
df.columns

['age', 'name']

In [12]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [13]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



Some data types make it easier to infer schema (like tabular formats such as csv which we will show later). 

However you often have to set the schema yourself if you aren't dealing with a .read method that doesn't have inferSchema() built-in.

Spark has all the tools you need for this, it just requires a very specific structure:

In [14]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

Next we need to create the list of Structure fields
    * :param name: string, name of the field.
    * :param dataType: :class:`DataType` of the field.
    * :param nullable: boolean, whether the field can be null (None) or not.

In [16]:
data_schema = [StructField("age", IntegerType(), True),StructField("name", StringType(), True)]

In [17]:
final_struc = StructType(fields=data_schema)

In [18]:
df = spark.read.json('people.json', schema=final_struc)

In [19]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Grabbing the data

In [20]:
df['age']

Column<b'age'>

In [21]:
type(df['age'])

pyspark.sql.column.Column

In [22]:
df.select('age')

DataFrame[age: int]

In [23]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [24]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [25]:
# Returns list of Row objects
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

Multiple Columns:

In [26]:
df.select(['age','name'])

DataFrame[age: int, name: string]

In [27]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Creating new columns

In [28]:
# Adding a new column with a simple copy
df.withColumn('newage',df['age']).show()

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [29]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [30]:
# Simple Rename
df.withColumnRenamed('age','supernewage').show()

+-----------+-------+
|supernewage|   name|
+-----------+-------+
|       null|Michael|
|         30|   Andy|
|         19| Justin|
+-----------+-------+



More complicated operations to create new columns

In [31]:
df.withColumn('doubleage',df['age']*2).show()

+----+-------+---------+
| age|   name|doubleage|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|       60|
|  19| Justin|       38|
+----+-------+---------+



In [32]:
df.withColumn('add_one_age',df['age']+1).show()

+----+-------+-----------+
| age|   name|add_one_age|
+----+-------+-----------+
|null|Michael|       null|
|  30|   Andy|         31|
|  19| Justin|         20|
+----+-------+-----------+



In [33]:
df.withColumn('half_age',df['age']/2).show()

+----+-------+--------+
| age|   name|half_age|
+----+-------+--------+
|null|Michael|    null|
|  30|   Andy|    15.0|
|  19| Justin|     9.5|
+----+-------+--------+



In [34]:
df.withColumn('half_age',df['age']/2)

DataFrame[age: int, name: string, half_age: double]

### Using SQL

To use SQL queries directly with the dataframe, you will need to register it to a temporary view:

In [35]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")

In [36]:
sql_results = spark.sql("SELECT * FROM people")

In [37]:
sql_results

DataFrame[age: int, name: string]

In [38]:
sql_results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [39]:
spark.sql("SELECT * FROM people WHERE age=30").show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [40]:
# DataFrame approach
df.filter(df.age == 30).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



We won't really be focusing on using the SQL syntax for this course in general, but keep in mind it is always there for you to get you out of bind quickly with your SQL skills!

Alright that is all we need to know for now!